In [140]:
import pandas as pd
warf = pd.read_csv('data/warfarin.csv')

In [141]:
warf = warf.fillna('Unknown')

In [142]:
to_change = list(warf)
to_change.remove('Height (cm)')
to_change.remove('Weight (kg)')
for col in to_change:
    # get list of keys and convert to numbers
    key_list = warf[col].unique().tolist()
    iter_i = iter(range(len(key_list)))
    total_map = {i: iter_i.__next__() for i in key_list}
    # reassign vals
    warf[col] = warf[col].map(total_map)

In [143]:
warf

,PharmGKB Subject ID,Gender,Race,Ethnicity,Age,Height (cm),Weight (kg),Indication for Warfarin Treatment,Comorbidities,Diabetes,...,VKORC1 -1639 consensus,VKORC1 497 consensus,VKORC1 1173 consensus,VKORC1 1542 consensus,VKORC1 3730 consensus,VKORC1 2255 consensus,VKORC1 -4451 consensus,Unnamed: 63,Unnamed: 64,Unnamed: 65
0,0,0,0,0,0,193.04,115.7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,1,176.53,144.2,0,1,0,...,1,0,1,1,1,1,1,0,0,0
2,2,1,0,0,2,162.56,77.1,0,0,0,...,2,1,0,2,0,0,0,0,0,0
3,3,0,0,0,0,182.24,90.7,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,4,0,0,0,1,167.64,72.6,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,5,0,0,0,2,177.8,104.3,0,0,0,...,2,1,0,2,0,0,0,0,0,0
6,6,0,0,0,3,167.64,84.8,0,0,0,...,2,1,0,2,2,0,0,0,0,0
7,7,0,0,0,2,187.96,99.8,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8,8,1,0,0,0,160.02,106.6,0,1,0,...,2,1,0,2,2,0,0,0,0,0
9,9,0,0,0,0,177.8,93.9,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [145]:
warf.to_pickle('data/cleaned_warfarin.pkl')